In [1]:
!pip install sdv
!pip install pandas
!pip install scikit-learn
!pip install io
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 3.11.0
    Uninstalling platformdirs-3.11.0:
      Successfully uninstalled platformdirs-3.11.0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.35.23
    Uninstalling botocore-1.35.23:
      Successfully uninstalled botocore-1.35.23
  Attempting uninstall: s

In [2]:
import kagglehub
from sdv.single_table import CTGANSynthesizer
import pandas as pd
import numpy as np
from sklearn import preprocessing 
from io import StringIO
from collections import defaultdict

from sdv.metadata import Metadata
from sdv.evaluation.single_table import evaluate_quality


In [3]:
# playground_series_s4e11_path = kagglehub.competition_download('playground-series-s4e11')

In [4]:
path='/kaggle/input/playground-series-s4e11/train.csv'
train = pd.read_csv(path)


In [5]:
student_train = train[train["Working Professional or Student"]=="Student"]
student_train.drop(['id','Working Professional or Student','Profession','Work Pressure','Job Satisfaction'],axis=1, inplace=True)
student_train.dropna(axis=1, how='all', inplace=True)

In [6]:
string_columns = student_train.select_dtypes(include=['object','string']).columns
numeric_student = student_train[student_train.columns.difference(string_columns)]


In [7]:
string_student=student_train[string_columns]

In [8]:

d = defaultdict(preprocessing.LabelEncoder)
string_student = string_student.apply(lambda x: d[x.name].fit_transform(x))

In [9]:
numeric_student = numeric_student.join(string_student)

In [10]:
numeric_student.select_dtypes(include=['object']).columns

Index([], dtype='object')

In [11]:

original_means = numeric_student.mean()
original_stds = numeric_student.std()
original_corr = numeric_student.corr()

In [12]:
# Define metadata for your dataset
metadata = Metadata()
# md = metadata.detect_from_dataframe(student_train).save_to_json('./metadata.json')

In [13]:
metadata_path='/kaggle/input/student-metadata/metadata.json'

In [14]:
md = metadata.load_from_json(metadata_path)


In [15]:
synthesizer = CTGANSynthesizer(
    md,
    epochs=500,
    generator_dim=(256, 256, 256),
    log_frequency=True)



In [16]:
synthesizer.auto_assign_transformers(numeric_student)


In [17]:
synthesizer.fit(numeric_student)

In [18]:
synthesizer.save(
    filepath='/kaggle/working/my_synthesizer.pkl'
)

In [19]:
synthetic_data = synthesizer.sample(len(numeric_student))


In [20]:
synthetic_data.to_csv('/kaggle/working/numeric_synethetic_data.csv')

In [21]:
quality_score = evaluate_quality(synthetic_data, numeric_student,md)
print(f"Quality Score: {quality_score}")

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 15/15 [00:00<00:00, 102.17it/s]|
Column Shapes Score: 83.7%

(2/2) Evaluating Column Pair Trends: |██████████| 105/105 [00:00<00:00, 127.14it/s]|
Column Pair Trends Score: 72.52%

Overall Score (Average): 78.11%

Quality Score: <sdmetrics.reports.single_table.quality_report.QualityReport object at 0x7eeb6825ebf0>


In [22]:
synthesizer.get_loss_values()


,Epoch,Generator Loss,Discriminator Loss
0,0,1.579942,-0.043535
1,1,1.129351,-0.006719
2,2,0.841202,-0.070814
3,3,0.397571,-0.065605
4,4,0.142792,-0.128091
...,...,...,...
495,495,-0.253239,0.428573
496,496,-0.741080,0.219617
497,497,-0.000300,-0.483392
498,498,0.160666,-0.233624


In [23]:
fig = synthesizer.get_loss_values_plot()
fig.show()

In [24]:
# synthetic_data = synthetic_data.apply(lambda x: d[x.name].inverse_transform(x))


In [25]:
# synthetic_data.to_csv('/kaggle/working/results.csv')